In [ ]:
%%capture
!pip3 install seaborn

In [ ]:
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
dataset = pd.read_csv('/content/merge3doang.csv')

In [ ]:
dataset

,module_name,discussion_title,question
0,Mekanisme Belajar,menu DropDwon Tidak Muncul,"Halo rekan-rekan dicodingMohon bantuannya,Menu..."
1,Mekanisme Belajar,Responsibilitas Submission Kurang Jelas,"Dalam submission tertera 4 column, akan tetapi..."
2,Mekanisme Belajar,Cara penulisan array dan object yg bener yg ma...,"Teksnya:""Array adalah tipe data yang ditulis d..."
3,Mekanisme Belajar,Bagaimana cara menghasilkan output yang hanya ...,"Saya punya kode seperti berikut :var pesan = ""..."
4,Mekanisme Belajar,bisakah menggunakan jquery untuk submission?,bisakah menggunakan jquery untuk submission?
...,...,...,...
8827,Submission: Proyek Pertama,Kriteria 2: Styling dengan Sass,"Izin bertanya, apakah hanya dengan menggunakan..."
8828,Memulai dengan SCSS,info cek sass version yang benar,sass --version . jika mengikuti yang dimodul m...
8829,Latihan: Menerapkan Firebase Authentication,user Ketika register,"saya mempunyai masalah ketika,user berhasil re..."
8830,Memulai dengan SCSS,error kode,saat memasukkan syntak sass styles.scss di ter...


In [ ]:
dataset['question'] = dataset['question'].str.replace('<p>', '')
dataset['question'] = dataset['question'].str.replace('<h2>', '')
dataset['question'] = dataset['question'].str.replace('</h2>', '')
dataset['question'] = dataset['question'].str.replace('<h3>', '')
dataset['question'] = dataset['question'].str.replace('</h3>', '')
dataset['question'] = dataset['question'].str.replace('</p>', '')
dataset['question'] = dataset['question'].str.replace('<strong>', '')
dataset['question'] = dataset['question'].str.replace('</strong>', '')
dataset = dataset[dataset['question'].str.contains('<img src=')==False]
dataset=dataset.reset_index(drop=True)

In [ ]:
dataset

,module_name,discussion_title,question
0,Mekanisme Belajar,menu DropDwon Tidak Muncul,"Halo rekan-rekan dicodingMohon bantuannya,Menu..."
1,Mekanisme Belajar,Responsibilitas Submission Kurang Jelas,"Dalam submission tertera 4 column, akan tetapi..."
2,Mekanisme Belajar,Cara penulisan array dan object yg bener yg ma...,"Teksnya:""Array adalah tipe data yang ditulis d..."
3,Mekanisme Belajar,Bagaimana cara menghasilkan output yang hanya ...,"Saya punya kode seperti berikut :var pesan = ""..."
4,Mekanisme Belajar,bisakah menggunakan jquery untuk submission?,bisakah menggunakan jquery untuk submission?
...,...,...,...
8825,Submission: Proyek Pertama,Kriteria 2: Styling dengan Sass,"Izin bertanya, apakah hanya dengan menggunakan..."
8826,Memulai dengan SCSS,info cek sass version yang benar,sass --version . jika mengikuti yang dimodul m...
8827,Latihan: Menerapkan Firebase Authentication,user Ketika register,"saya mempunyai masalah ketika,user berhasil re..."
8828,Memulai dengan SCSS,error kode,saat memasukkan syntak sass styles.scss di ter...


In [ ]:
STOPWORDS = set(stopwords.words('indonesian'))
MIN_WORDS = 4
MAX_WORDS = 200

PATTERN_S = re.compile("\'s")  # matches `'s` from text  
PATTERN_RN = re.compile("\\r\\n") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace 

def clean_text(text):
    """
    Series of cleaning. String to lower case, remove non words characters and numbers.
        text (str): input text
    return (str): modified initial text
    """
    text = text.lower()  # lowercase text
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

def tokenizer(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True):
    """
    Lemmatize, tokenize, crop and remove stop words.
    """
    stemmer = WordNetLemmatizer()
    tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    tokens = ' '.join(tokens)
    return tokens    


def clean_sentences(df):
    """
    Remove irrelavant characters (in new column clean_sentence).
    Lemmatize, tokenize words into list of words (in new column tok_lem_sentence).
    """
    print('Cleaning sentences...')
    dataset['question'] = dataset['question'].apply(clean_text)
    dataset['question'] = dataset['question'].apply(
        lambda x: tokenizer(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True))
    return dataset
    
dataset = clean_sentences(dataset)

Cleaning sentences...


In [ ]:
dataset

,module_name,discussion_title,question
0,Mekanisme Belajar,menu DropDwon Tidak Muncul,halo rekan rekan dicodingmohon bantuannya menu...
1,Mekanisme Belajar,Responsibilitas Submission Kurang Jelas,dalam submission tertera 4 column akan tetapi ...
2,Mekanisme Belajar,Cara penulisan array dan object yg bener yg ma...,teksnya array adalah tipe data yang ditulis de...
3,Mekanisme Belajar,Bagaimana cara menghasilkan output yang hanya ...,saya punya kode seperti berikut var pesan saya...
4,Mekanisme Belajar,bisakah menggunakan jquery untuk submission?,bisakah menggunakan jquery untuk submission
...,...,...,...
8825,Submission: Proyek Pertama,Kriteria 2: Styling dengan Sass,izin bertanya apakah hanya dengan menggunakan ...
8826,Memulai dengan SCSS,info cek sass version yang benar,sas version jika mengikuti yang dimodul maka i...
8827,Latihan: Menerapkan Firebase Authentication,user Ketika register,saya mempunyai masalah ketika user berhasil re...
8828,Memulai dengan SCSS,error kode,saat memasukkan syntak sas style sc di terminal


# START

In [ ]:
import pandas
import scipy
import math
import csv

In [ ]:
dataset['question']

0       halo rekan rekan dicodingmohon bantuannya menu...
1       dalam submission tertera 4 column akan tetapi ...
2       teksnya array adalah tipe data yang ditulis de...
3       saya punya kode seperti berikut var pesan saya...
4             bisakah menggunakan jquery untuk submission
                              ...                        
8825    izin bertanya apakah hanya dengan menggunakan ...
8826    sas version jika mengikuti yang dimodul maka i...
8827    saya mempunyai masalah ketika user berhasil re...
8828      saat memasukkan syntak sas style sc di terminal
8829    pada poin submission ke 4 adakah yang punya re...
Name: question, Length: 8830, dtype: object

In [ ]:
query_sentence='java script'

In [ ]:
def run_sts_benchmark(batch,query_sentence):
  xx=[]
  xx.append(query_sentence)
  data = {'input': xx}  
  data_input = pd.DataFrame(data)
  cos_scores=[]
  for i in range(len(batch)):
    sts_encode1 = tf.nn.l2_normalize(embed(tf.constant(data_input['input'].tolist())), axis=1)
    sts_encode2 = tf.nn.l2_normalize(embed(tf.constant(dataset['question'][i:i+1])), axis=1)
    cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
    clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
    scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi
    cos_scores.append(scores)

  cos_scores_list=[]
  for i in range(len(cos_scores)):
    cos_scores_list_tensor=cos_scores[i].tolist()
    cos_scores_list.append(float(cos_scores_list_tensor[0]))
  
  max_cos_scores_list=[]
  a_list=[]
  b_list=[]

  for i in range(9):
    max_cos=max(cos_scores_list)
    max_cos_scores_list.append(max_cos)
    cos_scores_list.remove(max_cos)
    dataset_cos_max = dataset.loc[dataset['Cosines Similarity Scores'] == max_cos_scores_list[i]]
    a_list.append(dataset_cos_max['module_name'].values.tolist())
    b_list.append(dataset_cos_max['discussion_title'].values.tolist())
  a=[]
  b=[]
  for i in range(9):
    a.append(a_list[i][0])
    b.append(b_list[i][0])
  return a, b

In [ ]:
print('Selamat Datang di ChatBot DiskusAI!')
print('-----------------------------------')
print('-----------------------------------')

while True:
    text=str(input('Masukkan Pertanyaan Kamu (Ketik STOP Jika Ingin Mengakhiri Sesi) :'))
    if text=='STOP':
      print('Sampai Jumpa, Terima Kasih Telah Menggunakan DiskusAI! <3')
      break
    else:
      a, b =run_sts_benchmark(dataset,text)
      print('-----------------------------------')
      print('Berikut Modul yang Relevan dengan Pertanyaan Kamu: ')
      print('-----------------------------------')
      for j in range(len(a)):
        print(j+1,'.',a[j])
      print('-----------------------------------')
      print('-----------------------------------')
      print('Berikut Judul Diskusi yang Relevan dengan Pertanyaan Kamu: ')
      print('-----------------------------------')
      for k in range(len(b)):
        print(k+1,'.',b[k])
      print('-----------------------------------')

Selamat Datang di ChatBot DiskusAI!
-----------------------------------
-----------------------------------
Masukkan Pertanyaan Kamu (Ketik STOP Jika Ingin Mengakhiri Sesi) :java script
-----------------------------------
Berikut Modul yang Relevan dengan Pertanyaan Kamu: 
-----------------------------------
1 . Pengenalan Kelas
2 . Persetujuan Hak Cipta
3 . Mengunduh Starter Project, Membuat Berkas JavaScript, dan Menampilkan Todo pada Console
4 . Submission: Tugas Akhir Membuat Website
5 . Submission: Tugas Akhir Membuat Website
6 . Submission: Tugas Akhir Membuat Website
7 . Submission: Tugas Akhir Membuat Website
8 . Anatomi Website
9 . Rangkuman Materi
-----------------------------------
-----------------------------------
Berikut Judul Diskusi yang Relevan dengan Pertanyaan Kamu: 
-----------------------------------
1 . Kapan deadline menyelesaikan semua kelas program dasar pembelajaran java script?
2 . Web
3 . maksud dari attribute 'type' ?
4 . Pertanyaan terkait submission 
5 .